In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
from tensorflow.keras.models import load_model
model=load_model('model.h5')

In [4]:
with open('onehot_encoder_geo.pkl','rb') as f:
    onehot_encoder_geo = pickle.load(f)
    
with open('scaler.pkl','rb') as f:
    scaler = pickle.load(f)
    
with open("label_encoder_gender.pkl",'rb') as f:
    label_encoder_gender = pickle.load(f)

In [ ]:
# input data
input_data = {
    'CreditScore': 700,
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 76000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    'Geography': 'France'
}

In [7]:
## onehot encode Geography
geo_encoded =onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\dell\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,700,Male,35,5,76000.0,2,1,1,50000.0,France


In [10]:
input_df['Gender']= label_encoder_gender.transform(input_df[['Gender']])
input_df

c:\Users\dell\anaconda3\envs\myenv\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,700,1,35,5,76000.0,2,1,1,50000.0,France


In [11]:
input_df = pd.concat([input_df.drop(columns=['Geography'],axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,35,5,76000.0,2,1,1,50000.0,1.0,0.0,0.0


In [12]:
## Scaling
input_df_scaled = scaler.transform(input_df)
input_df_scaled

array([[ 0.47777958,  0.9335368 , -0.41329718, -0.00873285, -0.02948351,
         0.80419558,  0.63910148,  0.968496  , -0.87405312,  1.00467761,
        -0.58402209, -0.57427105]])

In [13]:
prediction = model.predict(input_df_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step


In [14]:
prediction_probe =prediction[0][0]
if prediction_probe > 0.5:
    print(f"The customer is likely to leave the bank with a probability of {prediction_probe:.2f}")
else:
    print(f"The customer is likely to stay with the bank with a probability of {1-prediction_probe:.2f}")

The customer is likely to stay with the bank with a probability of 0.97
